In [ ]:
!pip install torch torchvision prometheus_client --progress-bar off

In [ ]:
import numpy as np
import torch
import torchvision
import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from prometheus_client import CollectorRegistry, Gauge, push_to_gateway

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                                ])

trainset = datasets.MNIST('dataset/train',
                          download=True, train=True, transform=transform)
valset = datasets.MNIST('dataset/test',
                        download=True, train=False, transform=transform)

train_dl = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True)
val_dl = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=True)

model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(2, 2)),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2, 2)),
    nn.Flatten(),
    nn.Linear(in_features=5408, out_features=128),
    nn.Dropout(0.2),
    nn.Linear(in_features=128, out_features=10),
    nn.Softmax(dim=1)
)

print(model)


def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)


def fit(epochs, model, loss_func, opt, train_dl, val_dl):
    current_time = time.time()
    for epoch in range(epochs):
        step_counter = []
        time_pre_loop = time.perf_counter()
        model.train()
        for xb, yb in train_dl:
            time_pre_step = time.perf_counter()
            loss_batch(model, loss_func, xb, yb, opt)
            time_post_step = time.perf_counter()
            step_counter.append(time_post_step - time_pre_step)
        time_post_loop = time.perf_counter()
        step_time_avg = sum(step_counter) / len(step_counter)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in val_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print("Epoch: {epoch}. Epoch Time {time:0.2f}s,"
              " Step Time: {step:0.2f}ms Validation Loss: {val_loss}"
              .format(epoch=epoch, time=time_post_loop - time_pre_loop,
                      step=step_time_avg * 1000, val_loss=val_loss))

        value_formatter = "{0:.2f}"
        add_metrics_data(current_time, epoch, value_formatter.format(time_post_loop - time_pre_loop),value_formatter.format(step_time_avg * 1000))


pushgateway_url = "http://prom-push-as-pushgateway.apps.zero.massopen.cloud"
registry = CollectorRegistry()
epoch_gauge = Gauge(name='epoch_duration_seconds', documentation='epoch_value is the metric itself, the stuff in the {}s are tags',labelnames=["model","framework","date","epoch"],registry=registry)
step_gauge = Gauge(name='step_during_milliseconds', documentation='step_during_milliseconds is the metric itself, the stuff in the {}s are tags',labelnames=["model","framework","date","epoch"],registry=registry)

def add_metrics_data(current_time, epoch_num, epoch_value, step_value):                  
    epoch_gauge.labels("mnist-minimal","Pytorch",current_time,epoch_num).set(epoch_value)
    step_gauge.labels("mnist-minimal","Pytorch",current_time,epoch_num).set(step_value)  
    push_to_gateway(pushgateway_url, job='jupyterhub_load', registry=registry)


fit(epochs=5, model=model, loss_func=F.cross_entropy,
    opt=optim.Adam(model.parameters()), train_dl=train_dl, val_dl=val_dl)
